In [1]:
import os
import sys
projectDir = '/mnt/c/Users/Julien Fournier/Documents/GitHub/RakutenTeam'
sys.path.append(projectDir)

#Define path in the config file
import src.config as config
config.path_to_project = projectDir #directory of the project
config.path_to_data = os.path.join(projectDir, 'data', 'clean') #path to the data (dataframe)
config.path_to_results = os.path.join('/mnt/g/My Drive/DST/DST-Rakuten Project', 'results') #path to where the summary of the benchmark results will be saved (csv)
config.path_to_images = '/home/jul/DST/Rakuten/Data/images/image_train_resized' #path to the folder containing images
config.path_to_models = '/mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models' #path to the folder where the models will be saved
config.path_to_tflogs = os.path.join('/mnt/g/My Drive/DST/DST-Rakuten Project', 'tf_logs') #Path to where tensorboard logs will be saved

import pandas as pd
import numpy as np

import tensorflow as tf

from Rakuten_preprocessing import Rakuten_img_path

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier, MetaClassifier

from src.utils.saveload import load_classifier
from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

from sklearn.linear_model import LogisticRegression

import datetime


2024-03-11 16:00:18.968048: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 16:00:19.018090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 16:00:19.018119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 16:00:19.019523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 16:00:19.027679: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)
    
#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [3]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [4]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## Voting and stacking models

In [5]:
#Loading pre-trained model and specifying from_trained ansd epoch = 0 
# so that the voting classifier doesn't refit them
model_bert2 = load_classifier(name='bert-test')
model_bert2.lr_min = 1e-6
model_bert2.from_trained = 'bert-test'
model_bert2.epochs = 0

model_vit2 = load_classifier(name='vit-test')
model_vit2.lr_min = 1e-6
model_vit2.from_trained = 'vit-test'
model_vit2.epoch = 0

vot_clf = MetaClassifier(base_estimators=[('bert', model_bert2), ('vit', model_vit2)], meta_method='voting', voting='soft', weights=[0.5, 0.5])


# logi_clf = LogisticRegression(C=1)
# vot_clf = MetaClassifier(base_estimators=[('bert', model_bert2), ('vit', model_vit2)], final_estimator=logi_clf ,method='stacking', cv=5)

2024-03-11 16:00:31.111207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:00:31.111591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:00:31.166923: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:00:31.167019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:00:31.167070: I external/local_xla/xla/stream_executor

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading weights from  bert-test


/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



loading weights from  vit-test


In [13]:
backup_named_estimators_ = vot_clf.model.named_estimators_
estimators_backup = vot_clf.model.estimators_
# estimators_input_backup = vot_clf.model.estimators

In [69]:
print(backup_named_estimators_)
print(estimators_backup)
print(estimators_input_backup)

{'bert': TFbertClassifier(callbacks=[('EarlyStopping',
                             {'min_delta': 0, 'mode': 'max',
                              'monitor': 'accuracy', 'patience': 2,
                              'restore_best_weights': True, 'verbose': 1})],
                 epochs=0, from_trained='bert-test', lr_min=1e-06), 'vit': ImgClassifier(augmentation_params={'cval': 255, 'fill_mode': 'constant',
                                   'height_shift_range': 0.1,
                                   'horizontal_flip': True,
                                   'rotation_range': 20,
                                   'width_shift_range': 0.1},
              from_trained='vit-test', lr_min=1e-06, parallel_gpu=False)}
[TFbertClassifier(callbacks=[('EarlyStopping',
                             {'min_delta': 0, 'mode': 'max',
                              'monitor': 'accuracy', 'patience': 2,
                              'restore_best_weights': True, 'verbose': 1})],
                 epochs

In [16]:
vot_clf.model.named_estimators_.keys()
named_estimators_backup_ = vot_clf.model.named_estimators_
if name in vot_clf.model.named_estimators_.keys():
    vot_clf.model.named_estimators_[name] = None 

vot_clf.model.named_estimators_

NameError: name 'name' is not defined

In [11]:
from joblib import dump

# vot_clf.model.named_estimators_ = []
# vot_clf.model.estimators_ = []
# vot_clf.model.estimators = []
save_path = os.path.join(config.path_to_models, 'trained_models', 'voting-test')
dump(vot_clf.model, os.path.join(save_path, 'model.joblib'))

TypeError: cannot pickle 'weakref.ReferenceType' object

In [8]:
dir(vot_clf)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_predict_proba',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 '_validate_params',
 'base_estimators',
 'classes_',
 'classification_score',
 'cross_validate',
 'fit',
 'from_trained',
 'get_metadata_routing',
 'get_params'

In [10]:
vot_clf.model

VotingClassifier(estimators=[('bert', None), ('vit', None)], voting='soft',
                 weights=[0.5, 0.5])

In [7]:
# vot_clf.base_estimators = []
# dump(vot_clf, os.path.join(save_path, 'model.joblib'))
vot_clf.save('voting-test')
# dump(vot_clf, os.path.join(save_path, 'model.joblib'))

TypeError: cannot pickle 'weakref.ReferenceType' object

In [6]:
vot_clf.fit(X_train[:100], y_train[:100])

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading weights from  bert-test


/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



loading weights from  vit-test
Found 100 validated image filenames.


2024-03-11 15:53:45.495620: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-11 15:53:48.507669: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 15:53:49.203381: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 15:53:51.076537: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f623d336860 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 15:53:51.076592: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-11 15:53:51.076600: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-11 15:53:51.082804: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disa

4/4 [==============================] - 53s 616ms/step - loss: 4.2920 - accuracy: 0.0800 - lr: 5.0000e-05


MetaClassifier(base_estimators=[('bert',
                                 TFbertClassifier(callbacks=[('EarlyStopping',
                                                              {'min_delta': 0,
                                                               'mode': 'max',
                                                               'monitor': 'accuracy',
                                                               'patience': 2,
                                                               'restore_best_weights': True,
                                                               'verbose': 1})],
                                                  epochs=0,
                                                  from_trained='bert-test',
                                                  lr_min=1e-06)),
                                ('vit',
                                 ImgClassifier(augmentation_params={'cval': 255,
                                                                    'fill_mode': 'constant',
                                                                    'height_shift_range': 0.1,
                                                                    'horizontal_flip': True,
                                                                    'rotation_range': 20,
                                                                    'width_shift_range': 0.1},
                                               from_trained='vit-test',
                                               lr_min=1e-06,
                                               parallel_gpu=False))],
               voting='soft', weights=[0.5, 0.5])

In [ ]:
vot_clf.classification_score(X_train[:1000], y_train[:1000])

In [5]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_fusion_meta.csv'

#type of modality
modality = 'fusion'

#Type of classifier
class_type = 'MetaClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = 27
max_length = 256
n_epochs = 8
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_min=1e-6
lr_decay_rate = 0.8

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#name of previously saved models to use as base estimators
base_name_list = ['bert-test vit-test']

voting_type = 'soft'
voting_weights = [[0.5, 0.5]]

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
    #adding the set of parameters to the list
    params_list.append({'modality': modality,
                        'class': class_type,
                        'base_name': base_name,
                        'meta_method': 'voting',
                        'param_grid': {'voting': voting_type, 'weights': voting_weights},
                        'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                      })
  
#Running the batch over params_list
results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)

Fitting:  bert-test vit-test nan


2024-03-11 11:42:04.911754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 11:42:04.912289: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 11:42:04.983392: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 11:42:04.983489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 11:42:04.983539: I external/local_xla/xla/stream_executor

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading weights from  bert-test


...

2024-03-11 11:43:42.174100: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


531/531 [==============================] - 155s 276ms/step
Found 16984 validated image filenames.


2024-03-11 11:46:17.808634: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-11 11:46:20.803565: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


531/531 [==============================] - 140s 250ms/step
Test set, f1score:  0.20417604933572872


AttributeError: 'MetaClassifier' object has no attribute 'estimators_'

## Visualize tensorflow logs in tensorboard

In [ ]:
log_path = os.path.join(config.path_to_tflogs, 'camembert-base')

# Ensure the log_path is quoted to handle spaces
quoted_log_path = f'"{log_path}"'

%reload_ext tensorboard
%tensorboard --logdir {quoted_log_path}

Reusing TensorBoard on port 6006 (pid 3002456), started 0:00:05 ago. (Use '!kill 3002456' to kill it.)

## Example usage

In [ ]:
#defining callbacks
callbacks = []
def lrscheduler(epoch, lr):
  return max(1e-6, lr * 0.8)
callbacks = callbacks.append(('LearningRateScheduler', {'schedule': lrscheduler}))
callbacks = callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1, }))

clf = ImgClassifier(base_name='ResNet101', img_size=(224, 224, 3), num_class=27, drop_rate=0.2, epochs=5, batch_size=32, 
                    validation_data=(X_test, y_test), learning_rate=5e-5, callbacks=callbacks)

clf.fit(X_train, y_train)
clf.classification_score(X_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-10 13:07:20.850430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.850632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948545: I external/local_xla/xla/stream_executor

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu